# Getting Started with OpenHands SDK

This notebook is an introduction to building agents with the [OpenHands SDK](https://docs.openhands.dev/sdk).

**What you'll learn:**
1. How to create an LLM and Agent
2. How to use built-in tools (Terminal, FileEditor, TaskTracker)
3. **MCP integration** - Add Tavily search with just config (no code!)
4. **Built-in persistence** - Auto-save & resume with 2 parameters

**Why OpenHands SDK?**
- **MCP built-in** - Connect to any MCP server with config, not code
- **Persistence built-in** - No graph setup, no external DB, just 2 params
- Production-ready with type safety (Pydantic)
- Built-in tools for coding tasks
- Works with any LLM (OpenAI, Anthropic, etc.)

## Step 1: Install

In [ ]:
# Note: Tavily MCP requires Node.js (for npx command)
!pip install openhands-sdk python-dotenv -q

## Step 2: Load Environment

Create a `.env` file:
```
LLM_API_KEY=sk-your-openai-key
LLM_MODEL=openai/gpt-5
TAVILY_API_KEY=tvly-your-tavily-key
```

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

print(f"Model: {os.getenv('LLM_MODEL', 'openai/gpt-5')}")

## Step 3: Create the LLM

In [ ]:
from openhands.sdk import LLM

llm = LLM(
    model="openai/gpt-5",
    api_key=os.getenv("LLM_API_KEY"),
    base_url=os.getenv("LLM_BASE_URL", None),
)

print(f"✓ LLM ready: {llm.model}")

## Step 4: Configure MCP Server (Tavily)

[Model Context Protocol (MCP)](https://modelcontextprotocol.io/) lets you add tools via config instead of code.

OpenHands SDK has **built-in MCP support** - just pass a config dict to your Agent!

We'll use the [Tavily MCP Server](https://docs.tavily.com/documentation/mcp) which provides:
- `tavily-search` - Web search with citations
- `tavily-extract` - Extract content from URLs

**Requires:** Node.js installed (for `npx`)

In [ ]:
# MCP config - that's it! No custom classes needed.
# Compare this to ~40 lines of Action/Observation/Executor code!

mcp_config = {
    "mcpServers": {
        "tavily": {
            "command": "npx",
            "args": ["-y", "tavily-mcp@0.1.3"],
            "env": {"TAVILY_API_KEY": os.getenv("TAVILY_API_KEY")}
        }
    }
}

print("✓ MCP config ready (Tavily search + extract)")
print("  Tools will be: tavily-search, tavily-extract")

## Step 5: Create the Agent with MCP Tools

Pass `mcp_config` to the Agent - MCP tools are automatically discovered and added!

In [ ]:
from openhands.sdk import Agent, Tool
from openhands.tools.terminal import TerminalTool
from openhands.tools.file_editor import FileEditorTool
from openhands.tools.task_tracker import TaskTrackerTool

agent = Agent(
    llm=llm,
    tools=[
        Tool(name=TerminalTool.name),
        Tool(name=FileEditorTool.name),
        Tool(name=TaskTrackerTool.name),
        # Tavily tools come from MCP - no Tool() entry needed!
    ],
    mcp_config=mcp_config,  # MCP tools auto-discovered
)

print("✓ Agent ready")
print(f"  Built-in tools: {[t.name for t in agent.tools]}")
print(f"  MCP tools: tavily-search, tavily-extract (from Tavily MCP server)")

## Step 6: Create a Conversation with Persistence

OpenHands SDK has **built-in persistence** - just add 2 parameters:
- `persistence_dir`: Where to save (auto-creates directory)
- `conversation_id`: Unique session ID (use same ID to resume)

**Every event is auto-saved.** If interrupted, re-run with same ID to resume from exact point.

In [ ]:
from openhands.sdk import Conversation
import uuid

cwd = os.getcwd()

# Create a deterministic UUID from a name (so you can resume with same ID)
SESSION_NAME = "simple-agent-demo"
conversation_id = uuid.uuid5(uuid.NAMESPACE_DNS, SESSION_NAME)

# Built-in persistence: just 2 parameters!
# Compare to LangGraph which requires: StateGraph, nodes, edges, checkpointer class
conversation = Conversation(
    agent=agent,
    workspace=cwd,
    persistence_dir="./.conversations",     # Where to save state
    conversation_id=conversation_id,        # UUID (use same name to resume)
)

print(f"✓ Conversation ready in: {cwd}")
print(f"✓ Persistence: ./.conversations/{conversation_id.hex}/")

## Step 7: Run a Research Query (uses Tavily search!)

In [ ]:
# Ask the agent to research something - it will use tavily-search from MCP
conversation.send_message("""
Estimate the main profit drivers for NVIDIA in 2026.
Use web search to find current information, then summarize your findings.
""")
conversation.run()

## OpenHands SDK Advantages

### 1. MCP Integration (No Code!)

We added Tavily search with just a config dict - no custom classes:

```python
# OpenHands SDK: 10 lines of config
mcp_config = {
    "mcpServers": {
        "tavily": {"command": "npx", "args": ["-y", "tavily-mcp"], ...}
    }
}
agent = Agent(llm=llm, mcp_config=mcp_config)
```

vs. Custom tool approach (~40 lines): See `01_deep_research.ipynb` for the Action/Observation/Executor pattern when you need full control.

### 2. Built-in Persistence

Your conversation is saved to `.conversations/<uuid>/`. Resume anytime:

```python
import uuid
# Same name = same UUID = auto-restore
conversation_id = uuid.uuid5(uuid.NAMESPACE_DNS, "simple-agent-demo")
conversation = Conversation(
    agent=agent,
    persistence_dir="./.conversations",
    conversation_id=conversation_id,
)
conversation.run()  # Continues where it left off
```

### Compare to LangGraph

| Feature | OpenHands SDK | LangGraph |
|---------|---------------|-----------|
| **Add tools** | Config dict (MCP) | Code or MCP |
| **Persistence** | 2 params | Graph + checkpointer class |
| **Graph required?** | ❌ No | ✅ Yes |
| **Lines for basic agent** | ~15 | ~40-50 |

**OpenHands SDK: MCP + Persistence without the graph overhead.**

---

## Next Steps

- `01_deep_research.ipynb` - Multi-agent workflow with custom tools
- `02_parallel_research.ipynb` - Parallel sub-agent delegation